In [ ]:
# read credentials from file 
import pandas as pd
import json

credentials = pd.read_json('credentials.json')

In [ ]:
# establish ssh connection to the host 
import paramiko 
host = "artemis.uni.ma"
port = 22
username = credentials['user'][0]
password = credentials['pwd'][0]

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

# Complete Process
continuously monitor the bestblockhash - returns the tip of the blockchain

In [ ]:
def getBestBlockHash():
    # get best blockhash - returns the header hash of the most recent block on the best block chain
    stdin, stdout, stderr = ssh.exec_command("bitcoin-cli getblockchaininfo")
    blockchaininfo = json.loads(stdout.read().decode("utf-8"))
    bestblockhash = blockchaininfo['bestblockhash']
    return bestblockhash

In [ ]:
def getBlockHash(height):
    # retrieve block hash by height of block
    stdin, stdout, stderr = ssh.exec_command(f"bitcoin-cli getblockhash {height}")
    block_hash = stdout.read().decode("utf-8")[:-1]
    return block_hash


In [ ]:
def getblock(blockhash): 
    command = "bitcoin-cli getblock " + blockhash + " 2"
    stdin, stdout, stderr = ssh.exec_command(command)
    block = json.loads(stdout.read().decode("utf-8"))

    # filter relevant blockinfo
    from datetime import datetime, timezone
    ts_epoch = block['time']
    block_timestamp = datetime.fromtimestamp(ts_epoch)
    block_median_time = block_timestamp.strftime('%Y-%m-%dT%H:%M')
    block_date = block_timestamp.strftime('%Y-%m-%d')
    block_hash = block['hash']
    block_height = block['height']
    # handling genesis block
    if block_height != 0:
        previousblockhash = block['previousblockhash']

    # build json object
    data = {}
    data['block_hash'] = block_hash
    data['block_height'] = block_height
    data['block_median_time'] = str(block_median_time) # change timestamp to median_time
    data['block_date'] = str(block_date)
    if block_height != 0:
        data['previousblockhash'] = previousblockhash
        

    return data, block

# Transactions

In [ ]:
from datetime import datetime, timezone

def gettx(tx):
        
    command = "bitcoin-cli getrawtransaction " + tx['txid'] + " true"
    stdin, stdout, stderr = ssh.exec_command(command)
    #print("stdout: ", stdout.read().decode("utf-8")[:-1])

    # load data into json object rawtx
    rawtx = json.loads(stdout.read().decode("utf-8"))

    txdata = {}
    txdata['txid'] = tx['txid']
    txdata['block_hash'] = block['hash']
    ts_epoch = block['time']
    block_timestamp = datetime.fromtimestamp(ts_epoch)
    block_date = block_timestamp.strftime('%Y-%m-%d')
    txdata['block_date'] = str(block_date)


    addr = ""
    val = 0
    outSum = 0
    inSum = 0
    inputAddrObject = {}
    outputAddrObject = {}
    input_address_list = [] 
    output_address_list = []

    ## check coinbase 
    if 'coinbase' not in rawtx['vin'][0]: 
        for i in rawtx['vin']:  
            command = "bitcoin-cli getrawtransaction " + i['txid'] + " true"
            stdin, stdout, stderr = ssh.exec_command(command)
            inputtx = json.loads(stdout.read().decode("utf-8"))
            inputtx_json_data = json.dumps(inputtx, indent=4, sort_keys=False)
            val = int (inputtx['vout'][i['vout']]['value']*100000000)
            inSum += val 

            # input addresses
            for ia in inputtx['vout'][i['vout']]['scriptPubKey']['addresses']: 
                inputAddrObject['addr'] = ia
                inputAddrObject['val'] = val

                jInAddr = json.dumps(inputAddrObject)
                jsonInDict = json.loads(jInAddr)
                input_address_list.append(jsonInDict)

        # output addresses
        for o in rawtx['vout']:
            if o['scriptPubKey']['type'] != "nulldata": # handling OP_RETURN data - can be skipped
                outSum += o['value']
                for a in o['scriptPubKey']['addresses']:
                    outputAddrObject['addr'] = a
                    outputAddrObject['outNr'] = o['n']
                    outputAddrObject['val'] = int(o['value']*100000000)

                    jOutAddr = json.dumps(outputAddrObject)
                    jsonOutDict = json.loads(jOutAddr)
                    output_address_list.append(jsonOutDict)
    else: 
        for o in rawtx['vout']:   
            if o['scriptPubKey']['type'] != "nulldata": # handling OP_RETURN data - can be skipped
                outSum += o['value']
                for a in o['scriptPubKey']['addresses']:
                    outputAddrObject['addr'] = a
                    outputAddrObject['outNr'] = o['n']
                    outputAddrObject['val'] = int(o['value']*100000000)

        jOutAddr = json.dumps(outputAddrObject)
        jsonOutDict = json.loads(jOutAddr)
        output_address_list.append(jsonOutDict)
        inSum = int(outSum*100000000)
        inputAddrObject['addr'] = "coinbase"
        inputAddrObject['val'] = inSum

        jInAddr = json.dumps(inputAddrObject)
        jsonInDict = json.loads(jInAddr)
        input_address_list.append(jsonInDict)


    # get degrees
    txdata['outDegree'] = len(rawtx['vout'])
    txdata['inDegree'] = len(rawtx['vin'])



    x = round(outSum*100000000, 0)
    txdata['outSum'] = int( x)
    txdata['inSum'] = inSum
    txdata['input_list'] = input_address_list  
    txdata['output_list'] = output_address_list 
        
        
    return txdata